# Luentoviikko 3

Osallistuin viikon opetukseen katsomalla varsinaisen luennon videotallenteet sekä demon panoptosta. 

Päiväkirjan teossa käytettyä aineistoa:
- [Luento 3] (https://github.com/InfoTUNI/joda2022/blob/master/luentomuistio/luento03.ipynb)
- [Viikon demo] (https://github.com/InfoTUNI/joda2022/blob/master/koodiesimerkit/Session%203%20-%20Loan%20Status%20Prediction.ipynb)

### Koneoppimisen periaatteet

Opetetaan algoritmi koneoppimisen perusteiden mukaisesti
- Koneoppimisen 4 tyyppiä: ohjattu, ohjaamaton, (vahvistettu ja -syväoppiminen)

Ohjatun oppimisen työnkulku
- Raakadata (esim. luottokorttipäätökset)
- Label (ennustettava muuttuja, onko luottokortteja myönnetty vai ei), ei aina saatavilla, johdettava raakadatasta manuaalisesti
- Piirteet - joukko muuttujia, jotka ennustavat tulemaa (ikä, luottotiedto yms.)
- Malli muodostaa tulokset matriisista, opetetaan, luokitteluongelama, false negatives/positives, kumpi virhe kalliimpi (kortti myönnetään vai ei)
- Tulosten ennustaminen mallin perusteella, uusia piirteitä voidaan muodostaa koneoppimisen avulla

Piirteet
- binäärisiä, jatkuvia tai kategorisia
- muuttujilla oltava merkitystä, olennaiset piirteet erotettava datasta
- esim. "Punavuori" piirre ei ole hyödyllinen Berliinissä, jokin muu piirre ennustaa hintaa paremmin
- Tietokoneissa esim. piirteet jotka ennustavat hintaa (käyttis, näyttö, muisti...)
- piirteiden valinta riippuu mitä halutaan ennustaa (hinta, laitekehitys, palauteen laatu)
- ohjaamaton oppiminen, palautteiden klusterointi / ryhmittely
- laiteet jotka keräävät dataa 24/7, esim. verkkokamerat/sateliitit tallentavat vain piirteet tilan säästämiseksi, kaikkea ei haluta tallentaa

### Viisi oivallusta
- Koneoppimisalgoritmeja voidaan luokitella neljään eri tyyppiin.
- Algoritmien opetus vaatii laadukasta dataa
- Koneoppiminen vaatii ihmisen ohjausta datan esikäsittelyssä, jotta se on algoritmeille sopivaa
- Esikäsittelyä mm. muutujien standardointi ja turhan datan poisto
- Kuhunkin tutkimusongelmaan liittyvät piirteet (muuttujat) erotettava datasta

### Kehityskohteet
- En keksinyt tälle luentoviikolle valitettavaa, mutta mielestäni luennolla esitettiin paljon hyviä esimerkkejä.

### Koodidemo

In [21]:
import pandas as pd

# Haetaan data ja katsotaan miltä se näyttää
url = 'https://raw.githubusercontent.com/InfoTUNI/joda2022/master/koodiesimerkit/credit_train.csv'
data = pd.read_csv(url)

data.shape

(100514, 19)

In [10]:
# Tarkistetaan taulukon koko
data.shape

(100514, 19)

In [56]:
# Otetaan käyttöön 1000 riviä operaatioiden nopeuttamiseksi demossa
data = data[:100]

data.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15.0,0.0,253460.0,427174.0,0.0,0.0


In [74]:
# Poistetaan rivit, joilla laina on maksettu
activeLoans = data[data['Loan Status'] != 'Fully Paid']

activeLoans.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
5,89d8cb0c-e5c2-4f54-b056-48a645c543dd,4ffe99d3-7f2a-44db-afc1-40943f1f9750,Charged Off,206602.0,Short Term,7290.0,896857.0,10+ years,Home Mortgage,Debt Consolidation,16367.74,17.3,NaN,6.0,0.0,215308.0,272448.0,0.0,0.0
7,db0dc6e1-77ee-4826-acca-772f9039e1c7,018973c9-e316-4956-b363-67e134fb0931,Charged Off,648714.0,Long Term,NaN,NaN,< 1 year,Home Mortgage,Buy House,14806.13,8.2,8.0,15.0,0.0,193306.0,864204.0,0.0,0.0
16,c9a16a9d-8801-4430-b445-dbf9cf845e31,abb4c446-08ea-49ff-aeb8-5e1e9da673e7,Charged Off,653004.0,Long Term,NaN,NaN,7 years,Home Mortgage,Debt Consolidation,14537.09,20.5,NaN,9.0,0.0,302309.0,413754.0,0.0,0.0
20,150ebbad-ebed-441e-b70d-2f350ad7dca6,40f729c9-54c7-4768-9fb5-2fa41d074c48,Charged Off,317108.0,Long Term,687.0,1133274.0,8 years,Rent,Debt Consolidation,9632.81,17.4,53.0,4.0,0.0,60287.0,126940.0,0.0,0.0
22,c7e2b784-016b-4fc4-8fde-623f09126557,5b53e176-8fc7-48bf-9d78-ceb5aa284f36,Charged Off,153252.0,Short Term,714.0,1890690.0,2 years,Rent,Debt Consolidation,21900.35,15.7,NaN,12.0,0.0,891594.0,1081014.0,0.0,0.0


In [77]:
# Tallennetaan niiden asiakkaiden lainat joiden credit score on alle 1000
activeLoans = activeLoans[activeLoans['Credit Score'] < 1000]

activeLoans.head(5)

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
20,150ebbad-ebed-441e-b70d-2f350ad7dca6,40f729c9-54c7-4768-9fb5-2fa41d074c48,Charged Off,317108.0,Long Term,687.0,1133274.0,8 years,Rent,Debt Consolidation,9632.81,17.4,53.0,4.0,0.0,60287.0,126940.0,0.0,0.0
22,c7e2b784-016b-4fc4-8fde-623f09126557,5b53e176-8fc7-48bf-9d78-ceb5aa284f36,Charged Off,153252.0,Short Term,714.0,1890690.0,2 years,Rent,Debt Consolidation,21900.35,15.7,NaN,12.0,0.0,891594.0,1081014.0,0.0,0.0
32,feee8a68-5ddc-41a6-9d10-d5e3f2e4f7c6,1c9b6b3e-060d-4a70-8b08-522a7f589e89,Charged Off,130174.0,Short Term,733.0,524609.0,< 1 year,Rent,Debt Consolidation,9311.71,15.4,NaN,7.0,1.0,130701.0,268818.0,1.0,0.0
40,21d019a9-dc53-40e9-9c88-1ec3378283fe,64560eb4-f50d-4f14-8a86-b46c0381bef2,Charged Off,688468.0,Long Term,682.0,1494616.0,< 1 year,Rent,Debt Consolidation,14697.07,16.6,50.0,8.0,0.0,343995.0,843854.0,0.0,0.0
49,1ee5e9de-290c-45f0-8ef2-445c98afc6f6,1ee733fd-fda7-4666-bbcb-059103773627,Charged Off,456808.0,Long Term,598.0,1096167.0,10+ years,Own Home,Debt Consolidation,14341.39,14.1,NaN,8.0,0.0,161861.0,278058.0,0.0,0.0
